1. 利⽤爬網技術取得 【 591 房屋交易租屋網 】 中，位於 【 臺北及新北 】 的所有 【 租屋物件資料 】 

https://rent.591.com.tw/?kind=0&region=1

In [15]:
import json
import os
import pandas as pd
import re
import requests
import time

from bs4 import BeautifulSoup as bs
from lxml import etree
from pprint import pprint
from pyquery import PyQuery as pq
from selenium import webdriver
from urllib.parse import urlencode

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

In [16]:
index_url='https://rent.591.com.tw/'

In [17]:
client = requests.session()

In [18]:
res = client.get(index_url,headers=headers)

## 轉換到台北

In [38]:
res.cookies['urlJumpIp']='1'

In [47]:
res.cookies.get_dict()

{'591_new_session': 'eyJpdiI6IklFWFUwZEpLbjJzRTByUFo5NHVhUFE9PSIsInZhbHVlIjoicElscXV5NFwvVjRTTXQ0T3ZlOGtvUjdCdkVEOGVzZzFoK0JtbCtVczRvMWpNVWg4bkthWlZRMktxUlhLaVJFVjhIU0Vpek1CUHdGZEJ6WkxXdFgwaGRRPT0iLCJtYWMiOiI3OTUwNDBjMTc2NDE0MzJlMGUwYzIxMGEyMGMyYjEyZDFmNjIyZjBmNDYyNmYyNmMyZWU4MTM3M2IyNTlhZDdiIn0%3D',
 'PHPSESSID': 'ai4smg2vpbn2kmt9vtbunhhnh5',
 'new_rent_list_kind_test': '1',
 'urlJumpIp': '6',
 'urlJumpIpByTxt': '%E6%A1%83%E5%9C%92%E5%B8%82',
 'webp': '1'}

In [ ]:
soup = bs(res.text, "lxml")

In [ ]:
soup

In [ ]:
totalRows = soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext')[0]['data-total']

## 抓 csrf-token

In [ ]:
CSRF_TOKEN = [i['content'] for i in soup.select('head > meta[\'name\']') if i['name']=="csrf-token"][0]

In [ ]:
def get_searchUrl(region):
    params = {
        'is_new_list': 1,
        'type': 1,
        'kind':0,
        'searchtype':1,
        'region':region,
        'firstRow=':30,
        'totalRows':totalRows
    }
    base_url = 'https://rent.591.com.tw/home/search/rsList?'
    url = base_url + urlencode(params)
    return url

In [ ]:
search_url = get_searchUrl(1)

In [ ]:
headers['X-CSRF-Token'] = CSRF_TOKEN

In [ ]:
res_search = client.get(search_url,headers=headers)

In [ ]:
res_search.encoding = 'utf-8'

In [ ]:
data = json.loads(res_search.text)

In [ ]:
data.keys()

In [ ]:
len([i['id'] for i in data['data']['data']])

In [ ]:
hid = [i['id'] for i in data['data']['data']][0]

In [ ]:
detail_url = 'https://rent.591.com.tw/rent-detail-'+ str(hid) +'.html'

In [ ]:
res_detail = requests.get(detail_url,headers=headers)

In [ ]:
print(res_detail.text)

In [ ]:
soup_detail = bs(res_detail.text, "lxml")

In [ ]:
#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.userInfo > div.infoOne.clearfix > div.avatarRight > div 
> i

In [ ]:
#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.rightBox > div.detailInfo.clearfix > ul

In [ ]:
attr = [''.join(i.text.split()) for i in soup_detail.select('ul.attr > li')]

In [ ]:
#性別要求 男女生皆可
soup_detail.select('div.two')

In [ ]:
#main > div.main_house_info.clearfix > div.detailBox.clearfix > div.leftBox > ul.clearfix.labelList.labelList-1 > li:nth-child(7) > div.two > em

In [ ]:
out_rec={
    '出租者':soup_detail.select('div.avatarRight > div')[0].select('i')[0].text,
    '出租者身份':soup_detail.select('div.avatarRight > div')[0].text.split('（')[1].split('）')[0],
    '聯絡電話':soup_detail.select('span.dialPhoneNum')[0]['data-value'],
    attr[0].split(':')[0]:attr[0].split(':')[1],
    attr[1].split(':')[0]:attr[1].split(':')[1],
    attr[2].split(':')[0]:attr[2].split(':')[1],
    attr[3].split(':')[0]:attr[3].split(':')[1],
}

In [ ]:
out_rec